In [9]:
import os
from glob import glob

from pytmatrix import orientation, radar, tmatrix_aux, refractive
from pytmatrix.psd import PSDIntegrator
from pytmatrix.tmatrix import TMatrix, Scatterer

import numpy as np

import common
import processing

In [20]:
m_w_30C = {'S': complex(8.695, -2.971),
               'C': complex(8.511, -4.103)}
print(m_w_30C['S'])

radar_band = tmatrix_aux.wl_S
print(refractive.m_w_20C[radar_band])

(8.695-2.971j)
(8.876+0.653j)


In [18]:
def worker(site_name, band, temperature, cant):
    
    #refractive indices of water at 30C
    m_w_30C = {'S': complex(8.695, -2.971),
               'C': complex(8.511, -4.103)}
    

    
    # Main T-matrix parameters initialisation
    # Radar band in mm.
    if band == 'S':
        radar_band = tmatrix_aux.wl_S
    elif band == 'C':
        radar_band = tmatrix_aux.wl_C
    # Scatterer class from pytmatrix
    # tmatrix_aux.wl_C is the wavelength in mm
    # refractive.m_w_10C is a dictionnary containing the refractive index (at 10C) for different wavelength.
    if temperature == 0:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_0C[radar_band])
    elif temperature == 10:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_10C[radar_band])
    elif temperature == 20:
        scatterer = Scatterer(wavelength=radar_band, m=refractive.m_w_20C[radar_band])
    elif temperature == 30:   
        print(radar_band)
        scatterer = Scatterer(wavelength=radar_band, m=m_w_30C[band])
    else:
        print('temperature unknown')
        return None
            
    scatterer.or_pdf = orientation.gaussian_pdf(cant)
    scatterer.orient = orientation.orient_averaged_fixed    
    # PSDIntegrator classfrom pytmatrix
    scatterer.psd_integrator = PSDIntegrator()    
    # Defining the axis ratio of drops.
    scatterer.psd_integrator.D_max = 8
    scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)
    ##### !!!!HERE!!!! ######
    scatterer.psd_integrator.axis_ratio_func = common.bzv_model
    ##### !!!!HERE!!!! ######
    scatterer.psd_integrator.init_scatter_table(scatterer)
    
    if site_name == 'darwin':
        file_list = sorted(glob("/g/data/kl02/jss548/PST/disdrometer/Darwin/*.cdf"))
        dbz_array, zdr_array, kdp_array, att_array, time_array, rain_array = processing.darwin_disdro_to_radar_moments(file_list, scatterer)
        
    elif site_name == 'brisbane':
        file_list = sorted(glob('/g/data/kl02/jss548/PST/disdrometer/Brisbane/*.txt'))        
        dbz_array, zdr_array, kdp_array, att_array, time_array, rain_array = processing.brisbane_disdro_to_radar_moments(file_list, scatterer)
        
    elif site_name == 'broadmeadows':
        file_list = sorted(glob('/g/data/kl02/jss548/PST/disdrometer/Broadmeadows/*.csv'))
        dbz_array, zdr_array, kdp_array, att_array, time_array, rain_array = processing.broadmeadows_disdro_to_radar_moments(file_list, scatterer)
        
    elif site_name == 'mtview':
        file_list = sorted(glob('/g/data/kl02/jss548/PST/disdrometer/MtView/*.csv'))
        dbz_array, zdr_array, kdp_array, att_array, time_array, rain_array = processing.mtview_disdro_to_radar_moments(file_list, scatterer)
        
    else:
        print('unknown site')
        return None
    
    #masking
    dbz_mask1 = np.logical_or(np.isinf(dbz_array), np.isnan(dbz_array))
    dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)
    dbz_mask = np.logical_or(dbz_mask1, dbz_mask2)
    #apply filters
    dbz_array_filt = dbz_array[~dbz_mask]
    rain_array_filt = rain_array[~dbz_mask]
    zdr_array_filt = zdr_array[~dbz_mask]
    kdp_array_filt = kdp_array[~dbz_mask]
    att_array_filt = att_array[~dbz_mask]
    time_array_filt = time_array[~dbz_mask]
    #convert zdr into log space
    zdr_db_array_filt = 10*np.log10(zdr_array_filt)
    
    np.savez(f'pytmatrix_out/{site_name}_radarsim_{band}_{cant}deg_{temperature}C.npz', dbz_array=dbz_array_filt, rain_array=rain_array_filt, zdr_db_array=zdr_db_array_filt, kdp_array=kdp_array_filt, att_array=att_array_filt, time_array = time_array_filt)    
    print(f'Finished {site_name}_radarsim_{band}_{cant}deg_{temperature}C')
    
    return None

In [19]:
band_list = ['C', 'S']
temperature_list = [0, 10, 20, 30]
cant_list = [10]
site_list = ['darwin', 'brisbane', 'broadmeadows', 'mtview']


for band in band_list:
    for temperature in temperature_list:
        for cant in cant_list:
            for site in site_list:
                worker(site, band, temperature, cant)

print('complete')

processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060202.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060203.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060204.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060205.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060206.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060207.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060208.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060209.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060210.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdisdrometerC3.b1.20060211.000000.cdf
processing /g/data/kl02/jss548/PST/disdrometer/Darwin/twpdis

<ipython-input-18-45ef0822e27c>:63: RuntimeWarning: invalid value encountered in less
  dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)
<ipython-input-18-45ef0822e27c>:63: RuntimeWarning: invalid value encountered in greater
  dbz_mask2 = np.logical_or(dbz_array<-25, dbz_array>65)


Finished broadmeadows_radarsim_C_10deg_0C
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2017_12_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_01_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_02_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_03_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_04_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_05_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_06_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_07_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_08_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_09_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_10_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_11_MtView.csv
processing /g/data/kl02/jss548/PST/disdrometer/MtView/2018_12_MtView.csv
processin